# Frontier Models

This section uses AWS Bedrock to run inference tests using the Claude and Open AI models. 

In [ ]:
!pip -q install langchain_aws

In [ ]:
%package install boto3=1.34.51 langchain 

In [1]:
import json
import boto3
import botocore
import datetime
from tqdm import tqdm

from prompts import SYSTEM_PROMPTS
from IPython.display import Markdown, display


In [2]:
# boto3 must be version 1.34.51 or higher
boto3.__version__

'1.34.51'

## Initial Bedrock Test
This is an initial run of Bedrock with Anthropic to see how it responds to the financial analysis task

In [3]:
# Bedrock client initialise
config = botocore.config.Config(read_timeout=1000)
boto3_bedrock = boto3.client('bedrock-runtime',config=config)
# set the model
model = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
# load prompts into memory
with open('Data/prompts.json', 'rb') as f:
    prompts = json.load(f)

In [ ]:
# load the first prompt as a test. Use the chain of thought system prompt
prompt = f"{SYSTEM_PROMPTS['CoTDetailed_Claude']['prompt']} context: {prompts[0]['prompt'][1]['content']}"
prompt

In [ ]:
def setup_claude_request(prompt: str) -> str:
    
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 5000,
        "temperature": 0.7,
        "messages": [
            {
                "role":"user",
                "content": [{"type":"text", "text":prompt}]
            }
        ]
    }
    
    return json.dumps(native_request)

In [ ]:
GUARDRAIL_ID = os.environ['BEDROCK_GUARDRAIL_ID']
GUARDRAIL_VERSION = os.environ['BEDROCK_GUARDRAIL_VERSION']

model_id = model


def invoke_model_claude(prompt: dict, model_id: str) -> str:
    """Invoke the model using Bedrock. This is specifically designed for Anthropic models"""
    accept = 'application/json'
    contentType = 'application/json'

    # set up the request
    request = setup_claude_request(prompt)
    try:
        
        response = boto3_bedrock.invoke_model(
            body=request, 
            modelId=model_id, 
            accept=accept, 
            contentType=contentType,
            trace = "ENABLED"
        )
        response_body = json.loads(response.get('body').read())
        return response_body.get('content')[0].get('text') #Anthropic response template
    
    except botocore.exceptions.ClientError as error:
        
        if error.response['Error']['Code'] == 'AccessDeniedException':
               print(f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
            
        else:
            raise error

In [ ]:
response = invoke_model_claude(prompt, model_id)

In [ ]:
display(Markdown(response))

## Langgraph version
This is the langgraph version in prep for multi-agentic AI system

### Run Inference Loop for Dow Jones

In [4]:

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.rate_limiters import InMemoryRateLimiter

from langchain_aws import ChatBedrock
from pydantic import BaseModel, Field

import concurrent.futures

In [5]:
# set up the LLM in Bedrock
rate_limiter = InMemoryRateLimiter(
    requests_per_second=5,
    check_every_n_seconds=1,
    max_bucket_size=10,
)

llm = ChatBedrock(
    client = boto3_bedrock,
    #model = "us.meta.llama3-1-70b-instruct-v1:0",
    model = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0',
    temperature = 0.5,#0.7,
    max_tokens=4000,
    rate_limiter = rate_limiter
)

In [9]:
system_prompt = SYSTEM_PROMPTS['BASE_CLAUDE']['prompt']
prompt_template = PromptTemplate.from_template(system_prompt)

In [10]:
def run_model(prompt: dict, llm: RunnableSequence) -> dict:
    prompt_in = prompt_template.format(financials=prompt['prompt'][1]['content'])
    output = llm.invoke(prompt_in)
    decision_dict = {
        'date': prompt['date'],
        'security': prompt['security'],
        'response': output
    }
    return decision_dict

In [11]:
#prompt_template.format(financials=prompts[0]['prompt'][1]['content'])
prompt_template

PromptTemplate(input_variables=['financials'], input_types={}, partial_variables={}, template="You are a financial analyst and must make a buy, sell or hold decision on a company based only on the provided datasets. Compute common financial ratios and then determine the buy or sell decision. Explain your reasons in less than 250 words. Provide a confidence score for how confident you are of the decision. If you are not confident then lower the confidence score. You must answer in a JSON format with a 'decision', 'confidence score' and 'reason'. Company financial statements: {financials} ")

In [12]:
run_model(prompts[0], llm)

{'date': '2020-02-06',
 'security': 'MMM UN Equity',
 'response': AIMessage(content='```json\n{\n  "decision": "Hold",\n  "confidence score": 0.65,\n  "reason": "The company shows concerning financial trends with declining profitability metrics. Operating income decreased by 34% from t-1 to t, and net income dropped by 39%. The gross margin has declined from 47.6% to 46.7%, while the debt-to-equity ratio increased from 1.68 to 1.80. However, there are some positive indicators: revenue grew slightly by 1.5% year-over-year, and the company maintains substantial cash reserves of $2.45B despite a significant reduction from t-1. The P/E ratio is approximately 8.2, which is relatively low and could indicate undervaluation. The stock price has declined about 20% over the observed period, potentially pricing in some of these negative trends. Given the mixed signals—deteriorating profitability but reasonable valuation and stable revenue—a hold recommendation is prudent until clearer directional

In [16]:
%%time
# This creates the initial response - run this in parallel with 100 executors
data = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(run_model, prompt, llm) for prompt in prompts]
    data = [f.result() for f in futures]

ConnectionClosedError: Connection was closed before we received a valid response from endpoint URL: "https://bedrock-runtime.us-east-1.amazonaws.com/model/us.anthropic.claude-3-7-sonnet-20250219-v1%3A0/invoke".

In [ ]:
len(data)

In [ ]:
cleaned_output = [{'date': i['date'], 
             'security': i['security'], 
             'response': i['response'].content} 
            for i in data]

In [ ]:
with open('Data/Results - Bedrock.json', 'w') as f:
    json.dump(cleaned_output, f)

## Convert to JSON format
This is needed to run the trade analytics and compare to other LLMs

In [ ]:
# STEP 1: Set up the structured output
class FinancialOutput(BaseModel):
    """Answer to the users question along with justification"""
    decision: str = Field(..., description="Final Decision: BUY, SELL or HOLD")
    reason: str = Field(..., description="Summary of the final decision")
    confidence: int = Field(..., description="How confident you are of the decision")

In [ ]:
# STEP 2: set up the system prompt and set the structured output format
system_prompt = "You are an assistant to a financial analyst. You are responsible for formatting the documents that the analyst produces into a machine readable format. Use only the information provided in the context. Convert it into the structured output. Do not add anything to the analysts report and do not change the recommendation. Do not hallucinate. Find the investment decision. Find the conclusion. Add all of the wording of the thought process into the steps section. context: {context}"

prompt_template = PromptTemplate.from_template(system_prompt)
structured_llm = llm.with_structured_output(FinancialOutput)


In [ ]:
# STEP 3: Set up the function to call in each thread
def run_cleanup(prompt: dict, llm: RunnableSequence) -> dict:
    thought_process = prompt['response']
    prompt_in = prompt_template.format(context=thought_process)
    response = llm.invoke(prompt_in)
    final_dict = {
        'date': prompt['date'],
        'security': prompt['security'],
        'response': {
            'decision': response.decision,
            'reason': response.reason,
            'confidence': response.confidence,
            'thought_process': thought_process
        }
    }
    return final_dict

In [ ]:
# STEP 4: 
#%% time
responses = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(run_cleanup, prompt, structured_llm) for prompt in cleaned_output]
    responses = [f.result() for f in futures]

In [ ]:
# STEP 5: convert to the correct format
final_json = {
    'run_date': str(datetime.datetime.now()),
    'system_prompt': SYSTEM_PROMPTS['CoTDetailed_Claude']['prompt'],
    'dataset': 'data_quarterly_pit_indu',
    'model': model,
    'results': responses
}

In [ ]:
# STEP 6: Save the 
with open(f'Results/results - {str(datetime.datetime.now())}.json', 'w') as f:
    json.dump(final_json, f)